In [1]:
import sqlite3
import pandas as pd

In [2]:
db_file=r'reddit_comments.db'
conn = sqlite3.connect(db_file)

In [3]:
input=pd.read_sql("SELECT * FROM rc WHERE pcomment IS NOT NULL LIMIT 20",conn)

In [4]:
for index,row in input.iterrows():
    print('parent: ',row['pcomment'])
    print('comment: ',row['comment'])
    print('#######################')

parent:  Yeah, probably. 
comment:  If she can't say yes on the spot, I'd legit reconsider the proposal. 
#######################
parent:  I legitimately have no idea what's going on, but am okay with it.  Also someone changed my text flair &amp;#3232;_&amp;#3232;
comment:  Me neither
#######################
parent:  And sorta not really even a doctor.
comment:  He was a great Dr. Just not for humans.  
#######################
parent:  So, fat dudes and feminists wanted to circlejerk in person?
comment:  That's offensive to any normal fat guy.
#######################
parent:  If that epiphany ever comes, we probably won't see a rant about it.
comment:  We'll see a lack of rants.
#######################
parent:  'sonuvabitch' -vapingwithtwisted420  That compilation video someone posted had me in tears. 
comment:  Oh I know I too was dying. Now every time he says that in a video I crack up!
#######################
parent:  Between *Dawn*, *New Horizons*, *Akatsuki*, the end of MESSENGER,

In [5]:
input=pd.read_sql("SELECT * FROM rc WHERE pcomment IS NOT NULL LIMIT 10000",conn)

In [6]:
#converting text to lowercase
def lower_txt(sentence):
    return sentence.lower()
input['pcomment']=input['pcomment'].apply(lower_txt)
input['comment']=input['comment'].apply(lower_txt)

In [7]:
input.head()

,parent_id,comment_id,pcomment,comment,score,subreddit
0,t1_cnbkf1f,t1_cnbn4j0,"yeah, probably.","if she can't say yes on the spot, i'd legit re...",11,AskMen
1,t1_cnbkla9,t1_cnbn4j6,"i legitimately have no idea what's going on, b...",me neither,3,twitchplayspokemon
2,t1_cnblwj8,t1_cnbn4jc,and sorta not really even a doctor.,he was a great dr. just not for humans.,4,WTF
3,t1_cnbhvpr,t1_cnbn4jj,"so, fat dudes and feminists wanted to circleje...",that's offensive to any normal fat guy.,9,TumblrInAction
4,t1_cnbmy2p,t1_cnbn4js,"if that epiphany ever comes, we probably won't...",we'll see a lack of rants.,9,anime


In [8]:
#tokenizing text using treebank tokenizer
from nltk.tokenize import TreebankWordTokenizer

def token(sentence):
    sentence = TreebankWordTokenizer().tokenize(sentence)
    sentence = ' '.join(sentence)
    return sentence

input['pcomment']=input['pcomment'].apply(token)
input['comment']=input['comment'].apply(token)

In [9]:
input.head()

,parent_id,comment_id,pcomment,comment,score,subreddit
0,t1_cnbkf1f,t1_cnbn4j0,"yeah , probably .","if she ca n't say yes on the spot , i 'd legit...",11,AskMen
1,t1_cnbkla9,t1_cnbn4j6,"i legitimately have no idea what 's going on ,...",me neither,3,twitchplayspokemon
2,t1_cnblwj8,t1_cnbn4jc,and sorta not really even a doctor .,he was a great dr. just not for humans .,4,WTF
3,t1_cnbhvpr,t1_cnbn4jj,"so , fat dudes and feminists wanted to circlej...",that 's offensive to any normal fat guy .,9,TumblrInAction
4,t1_cnbmy2p,t1_cnbn4js,"if that epiphany ever comes , we probably wo n...",we 'll see a lack of rants .,9,anime


In [10]:
import collections
parent_word_counter = collections.Counter([word for sentence in input['pcomment'] for word in sentence.split()])
comment_words_counter = collections.Counter([word for sentence in input['comment'] for word in sentence.split()])

print('{} parent words.'.format(len([word for sentence in input['pcomment'] for word in sentence.split()])))
print('{} unique parent words.'.format(len(parent_word_counter)))
print('10 Most common words in the parent dataset:')
print('"' + '" "'.join(list(zip(*parent_word_counter.most_common(10)))[0]) + '"')
print()
print('{} comment words.'.format(len([word for sentence in input['comment'] for word in sentence.split()])))
print('{} unique comment words.'.format(len(comment_words_counter)))
print('10 Most common words in the comment dataset:')
print('"' + '" "'.join(list(zip(*comment_words_counter.most_common(10)))[0]) + '"')

177997 parent words.
20843 unique parent words.
10 Most common words in the parent dataset:
"the" "." "," "i" "a" "to" "it" "you" "and" "?"

162195 comment words.
19442 unique comment words.
10 Most common words in the comment dataset:
"." "the" "i" "," "a" "to" "it" "and" "that" "you"


In [11]:
#converting words to integeres from 1 to vocabulary length
from keras.preprocessing.text import Tokenizer

def tokenizer(data):
    tokenizer=Tokenizer(filters='')
    tokenizer.fit_on_texts(data)
    t=tokenizer.texts_to_sequences(data)
    return t, tokenizer

Using TensorFlow backend.


In [12]:
X,ptokens=tokenizer(input['pcomment'])

In [13]:
X[0:2]

[[138, 3, 166, 2],
 [4,
  2060,
  31,
  61,
  306,
  41,
  14,
  120,
  23,
  3,
  24,
  150,
  502,
  29,
  194,
  108,
  135,
  636,
  34,
  967,
  850,
  63,
  220,
  49,
  221,
  2061,
  49,
  1618,
  63,
  220,
  49,
  221,
  2061,
  49]]

In [14]:
Y,ctokens=tokenizer(input['comment'])

In [15]:
Y[0:2]

[[43, 103, 136, 17, 122, 159, 28, 2, 1310, 4, 3, 115, 1090, 6943, 2, 6944, 1],
 [44, 1013]]

In [16]:
#padding the parents and comments
from keras.preprocessing.sequence import pad_sequences
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    padding=pad_sequences(x,padding='post',maxlen=length)
    return padding

In [17]:
X=pad(X)
Y=pad(Y)

In [18]:
X

array([[ 138,    3,  166, ...,    0,    0,    0],
       [   4, 2060,   31, ...,    0,    0,    0],
       [   9, 2383,   36, ...,    0,    0,    0],
       ...,
       [   4,   90,  400, ...,    0,    0,    0],
       [   4,   45, 1426, ...,    0,    0,    0],
       [   4,  215,   30, ...,    0,    0,    0]], dtype=int32)

In [19]:
Y

array([[  43,  103,  136, ...,    0,    0,    0],
       [  44, 1013,    0, ...,    0,    0,    0],
       [  21,   20,    5, ...,    0,    0,    0],
       ...,
       [  32, 1520,    3, ...,    0,    0,    0],
       [  26,  125,   54, ...,    0,    0,    0],
       [   3,  213,   26, ...,    0,    0,    0]], dtype=int32)

In [20]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

In [21]:
Tx=X.shape[1]
Ty=Y.shape[1]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [22]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [23]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

W0628 06:13:01.918323 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [24]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context

In [25]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(ptokens.word_index), activation=softmax)

In [26]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, parent_vocab_size, comment_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    parent_vocab_size -- size of the python dictionary "human_vocab"
    comment_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx,1))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a,return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a,s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context,initial_state=[s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [27]:
model = model(Tx, Ty, n_a, n_s, len(ptokens.word_index), len(ctokens.word_index))

W0628 06:13:01.988569 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 06:13:02.004064 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 149, 1)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 149, 64)      8704        input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 149, 64)      0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [29]:
### START CODE HERE ### (≈2 lines)
opt = Adam(lr=0.005,beta_1=0.9,beta_2=0.999,decay=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
### END CODE HERE ###

W0628 06:13:23.774032 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 06:13:24.470161 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.



In [30]:
import numpy as np
m=X_train.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [43]:
outputs=Y_train.reshape(m,Ty,1).swapaxes(0,1)
outputs=list(outputs)

In [31]:
X_train=X_train.reshape(m,Tx,1)

In [48]:
model.fit([X_train, s0, c0], outputs, epochs=1, batch_size=100)

W0628 06:32:14.127678 140419835352832 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0628 06:33:44.993291 140419835352832 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
8000/8000 [==============================] - 1192s 149ms/step - loss: 306.7490 - dense_3_loss: 1.5548 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.0235 - dense_3_acc_2: 0.0581 - dense_3_acc_3: 0.0979 - dense_3_acc_4: 0.1360 - dense_3_acc_5: 0.1833 - dense_3_acc_6: 0.2350 - dense_3_acc_7: 0.2813 - dense_3_acc_8: 0.3291 - dense_3_acc_9: 0.3728 - dense_3_acc_10: 0.4141 - dense_3_acc_11: 0.4547 - dense_3_acc_12: 0.4906 - dense_3_acc_13: 0.5239 - dense_3_acc_14: 0.5565 - dense_3_acc_15: 0.5839 - dense_3_acc_16: 0.6105 - dense_3_acc_17: 0.6365 - dense_3_acc_18: 0.6595 - dense_3_acc_19: 0.6807 - dense_3_acc_20: 0.7025 - dense_3_acc_21: 0.7215 - dense_3_acc_22: 0.7428 - dense_3_acc_23: 0.7594 - dense_3_acc_24: 0.7729 - dense_3_acc_25: 0.7875 - dense_3_acc_26: 0.8042 - dense_3_acc_27: 0.8204 - dense_3_acc_28: 0.8316 - dense_3_acc_29: 0.8416 - dense_3_acc_30: 0.8520 - dense_3_acc_31: 0.8589 - dense_3_acc_32: 0.8677 - dense_3_acc_33: 0.8769 - dense_3_acc_34: 0.8835 - dense_3_acc_35: 0.8